# Dynamiques multiagents


> **Rappel**: si la visualisation (en particulier des commandes LaTeX) est défaillante, ouvrir le même fichier dans le [viewer jupyter](https://nbviewer.jupyter.org/github/nmaudet/teaching-iaro/blob/master/3-dynamiquesMultiagents/DynamiquesMultiagents.ipynb)

## Caractérisation des stratégies des agents

Reprenons les stratégies envisagées dans le cadre du dilemme du prisonnier. 
On voit que l'on peut les classer selon deux critères: 

* le **niveau d'information** requis par la règle: on parle d'une règle de niveau $k$ lorsque les $k$ derniers coups de l'adversaire sont requis. 
* le fait que la règle soit **déterministe ou stochastique**

On peut parler d'apprentissage dès qu'une règle utilise la connaissance des coups joués par les autres joueurs, autrement dit dès qu'elle est de niveau $k\geq 1$. 

> Pouvez-vous caractériser selon ces critères les règles random, tit-for-tat, toujours trahir...





## Meilleures réponses avec plusieurs agents

On a vu dans les cours précédents la notion de meilleure réponse. 
Remarquons tout d'abord que cette notion s'étend tout à fait naturellement à plus de deux agents. 


Lorsque le système multiagent comporte plus de deux agents, on peut construire de manière similaire les tables de meilleures réponses, mais selon les possibilités de stratégies de **tous** les autres agents. Par exemple, pour décrire les meilleures réponses d'un agent x aux actions de $x_1, x_2, x_3$, on aura une table de ce type: 

| BR(x)  | $x_1$  | $x_2$ | $x_3$ |
|------|------|------|------|
| acheter  | emprunter| emprunter  | emprunter |
| emprunter  | acheter| acheter  | emprunter |
| ... | ... | ... | ... |




**Problème:** la table de meilleure réponse grandit exponentiellement avec le nombre d'agents. 


Toutefois, dans de nombreuses situations, les actions des agents n'ont que des conséquences locales. On peut donc limiter la représentation des meilleures réponses aux agents concernés (par exemple aux voisins sur un graphe). On parle alors de **jeux graphiques**. 

**Exemple.** Imaginons par exemple que Riri, Fifi, et Loulou habitent dans une rue: Riri au début de la rue, Fifi au milieu de la rue, et Loulou à la fin de la rue. Les trois amis se demandent s'ils doivent acheter une tondeuse à gazon ou plutôt l'emprunter à leur voisin. Cette décision dépend du choix de leur voisin. Riri est voisin de Fifi, Fifi est voisin de Riri et Loulou, et Loulou n'est voisin que de Riri. 

In [12]:
def bestResponse(s,agent):
    if agent == 'riri' or agent=='loulou':
        if s['fifi']=='achete':
            return 'emprunte'
        else:
            return 'achete'
    elif agent == 'fifi':
        if s['riri']=='emprunte' and s['loulou']=='emprunte':
            return 'achete'
        else: 
            return 'emprunte'

## 1. Dynamiques de meilleures réponses
On peut sur cette base étudier le processus dynamique qui résulte du fait que les agents jouent itérativement des meilleures réponses. 

De manière générale, on peut parler **partial best-response**: une partie de la population d'agents joue une meilleure réponse à l'état courant. On a donc deux cas limites: 

* la situation où **un seul agent** est considéré à chaque fois: les actions sont donc modifiée 
* la situation où **tous les agents** modifient leurs stratégies simultanément à chaque fois, en réponse à l'observation de l'état courant. 






Dans la version "agent unique", l'algorithme de dynamique de meilleure réponse prend donc la simple forme suivante: 

In [14]:
agents = ['loulou', 'riri', 'fifi']
s= {'riri': 'achete', 'fifi': 'achete', 'loulou': 'achete'} # vecteur de stratégies initiales
equilibrium = False
print (s)
while not equilibrium:
    equilibrium = True
    for i in agents:
        if s[i]!=bestResponse(s,i):
            s[i]=bestResponse(s,i)
            equilibrium=False
        print (s)
    
        
        
    

{'loulou': 'achete', 'riri': 'achete', 'fifi': 'achete'}
{'loulou': 'emprunte', 'riri': 'achete', 'fifi': 'achete'}
{'loulou': 'emprunte', 'riri': 'emprunte', 'fifi': 'achete'}
{'loulou': 'emprunte', 'riri': 'emprunte', 'fifi': 'achete'}
{'loulou': 'emprunte', 'riri': 'emprunte', 'fifi': 'achete'}
{'loulou': 'emprunte', 'riri': 'emprunte', 'fifi': 'achete'}
{'loulou': 'emprunte', 'riri': 'emprunte', 'fifi': 'achete'}


> Quelle est l'autre équilibre de Nash de ce jeu? Pouvez-vous trouver une séquence des agents menant (depuis le même état initial) à cet équilibre? 
> Que se passe-t-il sur cet exemple si on considère la dynamique où tous les agents modifient simultanément leurs stratégies?  

#### Quelques propriétés simples des dynamiques de meilleures réponses: 

* Lorsqu'un équilibre est atteint, il s'agit bien d'un équilibre de Nash (par définition). 
* Les dynamiques de meilleures réponses cyclent nécessairement lorsqu'il n'y a pas d'équilibre de Nash
* Les dynamiques de meilleures réponses peuvent cycler *même si il existe des équilibres de Nash* 
* Toutefois, sous certaines conditions, on peut montrer qu'elles convergent nécessairement vers des équilibres de Nash. 

## 2. Dynamiques de réponses améliorantes

Au lieu de supposer que les agents changent de stratégie en choisissant la meilleure (ou une des meilleures s'il en existe plusieurs) réponse améliorante, on peut supposer que les réponses sont seulement améliorantes (donc meilleures que la stratégie actuelle, sans être nécessairement *la* meilleure)

### Dynamiques de mariages stables

Reprenons le cadre des mariages que vous avez étudié en détail avec Bruno Escoffier, et essayons de définir ce que pourrait être une dynamique de meilleure réponse, ou de réponse améliorante. 

On rappelle qu'une paire instable est une paire d'agents qui préférent être ensemble qu'avec leur partenaire actuel. 
Dans notre cadre, on dit qu'on résoud satisfait une paire lorsque l'on associe les deux agents de la paire. Les anciens partenaires se trouvent alors non associés. 

Prenons le cadre suivant: 

* on part d'une situation initiale, c'est-à-dire un matching (éventuellement partiel) entre les agents de M et W. 
* on considère une paire d'agents instable $(m,w) \in M \times W$. 
* dans une dynamique de réponse améliorante, on satisfait cette paire instable.  
* dans une dynamique de meilleure réponse, on considère un agent impliqué dans cette paire instable (par exemple $m$), et on satisfait sa paire instable préférée parmi toutes les paires instables dans lesquelles il est impliqué.




Knuth (1976) a montré que la dynamique de réponse améliorante ne converge pas forcément. 
Son exemple original était le suivant: 

| women | men |
| ----- | ----- |
|  w1: m1, m3, m2 |  m1: w2, w1, w3 |
|  w2: m3, m1, m2 | m2: w1, w3, w2 |
|  w3: m1, m3, m2 | m3: w1, w2, w3 |

Vous avez vu en TD d'autres exemples de tels cycles (procédure des divorces successifs). 

Le même type de cyle peut être obtenu pour la dynamique de meilleure réponse (Ackerman et al., 2008). 

Voici l'exemple: 

| women | men |
| ----- | ----- |
|  w1: m2, m3, m1 | m1: w1, w3, w2 |
|  w2: m1, m2, m3 | m2: w2, w1, w3 |
|  w3: m3, m1, m2 | m3: w1, w2, w3 |

Un cycle est le suivant (le . représente le fait que le femme d'indice $i$ n'est pas en couple): $(.,w2-m2,w3-m3) \rightarrow (w1-m3,w2-m2,.) \rightarrow (w1-m3,w2-m1,.) \rightarrow (w1-m3,.,w3-m1) \rightarrow  (w1-m2,.,w3-m1)  \rightarrow(.,w2-m2,w3-m1)  \rightarrow (.,w2-m2,w3-m3)$


De là découlent de nombreuses autres questions naturelles: 

* peut-on toujours une séquence qui mène à un état stable? 
* si la dynamique est aléatoire, quelle est la probabilité d'atteindre un état stable? 
* combien de temps peut-il falloir pour atteindre un état stable? 
* si on part d'un état initial particulier (par exemple le matching vide), est-ce que cela peut changer les choses? 



## 3. Fictitious Play



Supposons maintenant que les agents apprennent plus spécifiquement le comportement des autres joueurs, et jouent la meilleure réponse à la prédiction qu'ils font du comportement des autres agents. 
Un des modèles les plus simples d'un tel type d'apprentissage a été proposé par Brown (1951) sous le nom de *fictitious play*. 

Il repose donc sur les phases suivantes: 

* prédiction des actions des autres joueurs
* choix de la meilleure réponse à la stratégie des autres agents
* observation des actions des autres joueurs, mise à jour de la prédiction

**(Standard) Fictitious Play**: 
Nous présentons ici le modèle dans un contexte où seuls deux agents sont en interaction à chaque fois, même si le système comprend plus d'agents. (Le modèle s'étend à des interactions impliquant plus d'agents, mais il faut alors faire des choix sur la manière de combiner les probabilités assignées aux différents agents). 

* Il faut d'abord que l'agent fasse une prédiction *a priori* sur l'action initiale de l'agent (cette prédiction peut être une distribution de proba, pas nécessairement un coup donné). 
* Chaque joueur tient à jour la fréquence de sélection des actions et joue sa meilleure réponse à ce modèle probabiliste. 

Ici la meilleure réponse est entendu en terme de gain espéré pour chacune de ses actions. 



#### Exemple: 

Plaçons nous du point de vue l'agent <span style="color:red">R</span>, qui est opposé à l'agent <span style="color:blue">B</span>. 
La table suivante donne les gains pour le joueur R. 

|<span style="color:red">R</span> \ <span style="color:blue">B</span> | c | d |
| ----|---- | ---- |
|**a** | 7 | 10 |
|**b** | 10 | 0 |


Supposons que la stratégie de B soit de jouer c et d à 50/50. 
Et que R ait comme a priori que B va presque certainement toujours jouer c ($P(c)=0.8$), qui provient d'un échantillon initial de 4 c et 1 d. 



|  #c | #d | P(c) | P(d) |  BR(R) | coup B |
| ----|---- | ---- | ---- | ---- | ---- |
|  4 | 1 | 0.8 | 0.2 |  b | c |
|  5 | 1 | 0.83 | 0.17 |  b | d | 
|  5 | 2 | 0.71 | 0.29 |  a | c | 

Pourquoi a est-elle la meilleure réponse à l'itération 3? 

On voit qu'à cette étape, R estime que B joue c et d avec une probabilité 50/50. 
Quelle est le gain espéré pour lui de jouer 
* l'action a: $0.71 \times 7 + 0.29 \times 10 = 7.86$
* l'action b: $0.71 \times 10 + 0.29 \times 0 = 7.1$

On a donc gain espéré pour a > gain espéré pour b. La meilleure réponse est de jouer a. 

Et ainsi de suite pour les itérations suivantes...

|  #c | #d | P(c) | P(d) |  BR(R) | coup B |
| ----|---- | ---- | ---- | ---- | ---- |
|  6 | 2 | 0.75 | 0.25 |  a | d |
|  6 | 3 | 0.66 | 0.33 |  a | c |
| ... | ... | ... | ... | ... | ... |

  
  


> Mais notons que l'apprentissage reste rudimentaire. Par exemple, si j'observe que l'autre joueur joue *alternativement* c et d, un agent utilisant fictitious play attribuera à la limite une probabilité 50/50 pour l'autre agent de jouer c ou d, alors que s'il avait reconnu la séquence, il serait capable d'obtenir un bien meilleur gain. 

In [30]:
actions = [('a','b'),('a','b')]
matGains ={'a':{'a':(3,2),'b':(0,0)},'b':{'a':(0,0),'b':(2,3)}}
prior = [{'a':2,'b':1},{'a':1,'b':2}]
expectedGain = {'a':0,'b':0}
nbJoueurs = 2
nbObservations = sum (prior[0].values())
nbIterations = 3

def fictitious(observations):
    actionsChosen =[]
    for i in range(nbJoueurs):
        expectedGain = {'a':0,'b':0}
        print ('joueur ',i)
        for actionMe in actions[0]:
            bestAction = actions[0][0]
            bestGain =0
            for actionOther in actions[1]: 
                expectedGain[actionMe] += (observations[i][actionMe]/nbObservations)*matGains[actionMe][actionOther][i]
            print ("gain espéré pour ",actionMe, expectedGain[actionMe])
            if expectedGain[actionMe]>bestGain:
                bestAction=actionMe
                bestGain=expectedGain[actionMe]
        actionsChosen+= bestAction
    return actionsChosen

observations = prior.copy()
for i in range(nbIterations):
    print (observations)
    nextAction0, nextAction1 = fictitious(observations)
    print (nextAction0,nextAction1)
    observations[0][nextAction1]+=1
    observations[1][nextAction0]+=1


#fictitious(prior)

[{'b': 1, 'a': 2}, {'b': 2, 'a': 1}]
joueur  0
gain espéré pour  a 2.0
gain espéré pour  b 0.6666666666666666
joueur  1
gain espéré pour  a 0.6666666666666666
gain espéré pour  b 2.0
b b
[{'b': 2, 'a': 2}, {'b': 3, 'a': 1}]
joueur  0
gain espéré pour  a 2.0
gain espéré pour  b 1.3333333333333333
joueur  1
gain espéré pour  a 0.6666666666666666
gain espéré pour  b 3.0
b b
[{'b': 3, 'a': 2}, {'b': 4, 'a': 1}]
joueur  0
gain espéré pour  a 2.0
gain espéré pour  b 2.0
joueur  1
gain espéré pour  a 0.6666666666666666
gain espéré pour  b 4.0
b b


#### Quelques propriétés des dynamiques fictitious play

Supposons à présent que les deux agents utilisent une dynamique de fictitious play. 
On voit que l'on peut atteindre deux types de situations d'équilibre: 
* des équilibres pour lesquels les probabilités sont concentrées sur une action unique (équilibre pur)
* des équilibres pour lesquels les probabilités sont réparties sur plusieurs actions (équilibre mixte)

On sait alors que: 

* tout équilibre pur atteint par fictitious play doit être un équilibre de Nash
* la dynamique n'atteint pas nécessairement de situation d'équilibre (même mixte)
* il existe plusieurs catégories de jeux pour lesquels la convergence est garantie, par exemple les jeux à somme nulle. 


## 4. Autres techniques d'apprentissage

Considérons le problème suivant: chaque week-end des agents souhaitent se rendre dans un bar pour écouter de la musique. Les agents (supposons qu'ils sont au nombre de 100) ont la préférence suivante: ils souhaitent aller au bar pour écouter la musique, mais en même temps qu'il y ait trop de monde, sinon ils ne peuvent plus écouter correctement de la musique. 

Plus précisément: 

* si moins de 60 % de la population se rend au bar, l'agent préfère aller au bar
* si plus de 60% de la population se rend au bar, l'agent aurait préféré rester chez lui. 

Ls agents se rendent le premier week-end au bar, ils observent la situation, puis révise leur stratégie pour le deuxième week-end, etc.

Il s'agit d'un problème connu sous le nom de *El-Farol problem*, un jeu de minorité. 

On s'interroge sur la règle de décision que peuvent utiliser les agens dans cette seituation. 



> Que se passe-t-il si tous les agents utilisent la même méthode déterministe pour décider s'ils vont au bar ou pas?

> Que se passe-t-il si tous les agents utilisent pile ou face pour décider s'ils vont au bar ou pas?


Arthur propose de faire reposer la décision des agents sur un **ensemble de prédicteurs**. 

* prédicteurs cycliques: même nombre de personnes qu'il y a *k* semaines
* 100 - le nombre de personnes de la semaine dernière
* prédicteur fixe: 67 personnes
* moyenne glissante sur les 4 dernières semaines
* la tendance sur les 8 dernières semaines évaluée avec une régression linéaire et la méthode des moindre carrés

Chaque prédicteur se voit ensuite associé un score, selon la qualité de sa prédiction (+1 si la prédiction trop-de-monde / pas-trop-de-monde concorde avec l'observation). 

Une bibliothèque de prédicteurs est créée, puis chaque agent reçoit un ensemble de prédicteurs qu'il aura à sa disposition. A chaque tour, les agents utiliseront leur prédicteur le plus performant jusqu'alors. 